In [27]:
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle as pkl
import warnings

In [28]:
warnings.filterwarnings('ignore')
df = pd.read_csv('./data/Prostate_Cancer.csv')
df

,id,diagnosis_result,radius,texture,perimeter,area,smoothness,compactness,symmetry,fractal_dimension
0,1,M,23,12,151,954,0.143,0.278,0.242,0.079
1,2,B,9,13,133,1326,0.143,0.079,0.181,0.057
2,3,M,21,27,130,1203,0.125,0.160,0.207,0.060
3,4,M,14,16,78,386,0.070,0.284,0.260,0.097
4,5,M,9,19,135,1297,0.141,0.133,0.181,0.059
...,...,...,...,...,...,...,...,...,...,...
95,96,M,23,16,132,1264,0.091,0.131,0.210,0.056
96,97,B,22,14,78,451,0.105,0.071,0.190,0.066
97,98,B,19,27,62,295,0.102,0.053,0.135,0.069
98,99,B,21,24,74,413,0.090,0.075,0.162,0.066


In [29]:
def convert(arr):
    new_pred =[]
    for i in arr:
        if i>0.5:
            new_pred.append(1)
        else:
            new_pred.append(0)
    return new_pred

In [30]:
def Ann(x_train,y_train,x_test,y_test,shape,opt,loss_function,ep):
    model= keras.Sequential([
        keras.layers.Dense(100,input_shape=(shape,),activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10,activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1,activation='sigmoid')
    ])
    model.compile(
        optimizer=opt,
        loss=loss_function,
        metrics=['accuracy']
    )
    model.fit(x_train,y_train,epochs=ep)
    y_pred = model.predict(x_test)
    new_pred =convert(y_pred)
    print(classification_report(y_test,new_pred))
    return model

In [31]:
samples_to_take=df['diagnosis_result'][df['diagnosis_result']=='B'].value_counts()
samples_to_take[0]

38

In [32]:
final_df_m =df[df['diagnosis_result']=='M']
final_df_b= df[df['diagnosis_result']=='B']

In [33]:
final_b =final_df_b.sample(62,replace=True)
final_b

,id,diagnosis_result,radius,texture,perimeter,area,smoothness,compactness,symmetry,fractal_dimension
96,97,B,22,14,78,451,0.105,0.071,0.190,0.066
21,22,B,17,24,60,274,0.102,0.065,0.182,0.069
66,67,B,12,13,60,269,0.104,0.078,0.172,0.069
76,77,B,10,17,88,559,0.129,0.105,0.240,0.066
5,6,B,25,25,83,477,0.128,0.170,0.209,0.076
...,...,...,...,...,...,...,...,...,...,...
50,51,B,19,25,75,428,0.086,0.050,0.150,0.059
84,85,B,25,21,77,443,0.097,0.072,0.208,0.060
21,22,B,17,24,60,274,0.102,0.065,0.182,0.069
66,67,B,12,13,60,269,0.104,0.078,0.172,0.069


In [34]:
final_df_m.shape

(62, 10)

In [35]:
data = pd.concat([final_df_m,final_b],axis='rows')
data

,id,diagnosis_result,radius,texture,perimeter,area,smoothness,compactness,symmetry,fractal_dimension
0,1,M,23,12,151,954,0.143,0.278,0.242,0.079
2,3,M,21,27,130,1203,0.125,0.160,0.207,0.060
3,4,M,14,16,78,386,0.070,0.284,0.260,0.097
4,5,M,9,19,135,1297,0.141,0.133,0.181,0.059
6,7,M,16,26,120,1040,0.095,0.109,0.179,0.057
...,...,...,...,...,...,...,...,...,...,...
50,51,B,19,25,75,428,0.086,0.050,0.150,0.059
84,85,B,25,21,77,443,0.097,0.072,0.208,0.060
21,22,B,17,24,60,274,0.102,0.065,0.182,0.069
66,67,B,12,13,60,269,0.104,0.078,0.172,0.069


In [36]:
data.drop('id',axis=1, inplace=True)


In [37]:
x =data.drop('diagnosis_result',axis=1)
y =data['diagnosis_result']


In [38]:
y_final = pd.get_dummies(y,drop_first=True)

In [39]:
x_train,x_test,y_train,y_test =train_test_split(x,y_final,test_size=0.2,stratify=y_final)

In [40]:
model= Ann(x_train,y_train,x_test,y_test,8,'adam','binary_crossentropy',10)

Epoch 1/10
4/4 [==============================] - 1s 5ms/step - loss: 29.1247 - accuracy: 0.5152
Epoch 2/10
4/4 [==============================] - 0s 5ms/step - loss: 18.4585 - accuracy: 0.5758
Epoch 3/10
4/4 [==============================] - 0s 4ms/step - loss: 17.2315 - accuracy: 0.5556
Epoch 4/10
4/4 [==============================] - 0s 4ms/step - loss: 15.4882 - accuracy: 0.5657
Epoch 5/10
4/4 [==============================] - 0s 4ms/step - loss: 20.2181 - accuracy: 0.5051
Epoch 6/10
4/4 [==============================] - 0s 5ms/step - loss: 13.9625 - accuracy: 0.6061
Epoch 7/10
4/4 [==============================] - 0s 4ms/step - loss: 14.0007 - accuracy: 0.5051
Epoch 8/10
4/4 [==============================] - 0s 5ms/step - loss: 12.0091 - accuracy: 0.5455
Epoch 9/10
4/4 [==============================] - 0s 6ms/step - loss: 12.7463 - accuracy: 0.5859
Epoch 10/10
4/4 [==============================] - 0s 5ms/step - loss: 14.4951 - accuracy: 0.5657
              precision    re